## Bases de datos

In [ ]:
# SQL (creación y consulta de base de datos relacional local)
!pip install sqlalchemy sqlite-utils

# Soporte para archivos Excel (.xlsx)
!pip install openpyxl pandas

!pip install fuzzywuzzy python-Levenshtein

In [ ]:
file_name = "food_cleaned.csv"
file_path = f"../data/csv/{file_name}"
import pandas as pd
import os
import sqlite3

if not os.path.exists(file_path):
    raise FileNotFoundError(f"Archivo no encontrado: {file_path}")

df = pd.read_csv(file_path)

# Conexión a una base de datos local SQLite
conn = sqlite3.connect("../comida.db")
df.to_sql("comida", conn, if_exists="replace", index=False)

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("../comida.db")
consulta_sql = pd.read_sql( 'SELECT * FROM comida LIMIT 5', conn)

In [3]:
consulta_sql

,category,description,alpha_carotene,beta_carotene,beta_cryptoxanthin,carbohydrate,cholesterol,choline,fiber,lutein_and_zeaxanthin,...,major_minerals.potassium,major_minerals.sodium,major_minerals.zinc,vitamins.vitamin_a_-_rae,vitamins.vitamin_b12,vitamins.vitamin_b6,vitamins.vitamin_c,vitamins.vitamin_e,vitamins.vitamin_k,calories
0,Milk,"Milk, human",0.0,7.0,0.0,6.89,14.0,16.0,0.0,0.0,...,51.0,17.0,0.17,61.0,0.05,0.011,5.0,0.08,0.3,71.10
1,Milk,"Milk, NFS",0.0,4.0,0.0,4.87,8.0,17.9,0.0,0.0,...,157.0,39.0,0.42,59.0,0.56,0.060,0.1,0.03,0.2,50.75
2,Milk,"Milk, whole",0.0,7.0,0.0,4.67,12.0,17.8,0.0,0.0,...,150.0,38.0,0.41,32.0,0.54,0.061,0.0,0.05,0.3,60.60
3,Milk,"Milk, low sodium, whole",0.0,7.0,0.0,4.46,14.0,16.0,0.0,0.0,...,253.0,3.0,0.38,29.0,0.36,0.034,0.9,0.08,0.3,61.38
4,Milk,"Milk, calcium fortified, whole",0.0,7.0,0.0,4.67,12.0,17.8,0.0,0.0,...,150.0,38.0,0.41,32.0,0.54,0.061,0.0,0.05,0.3,60.60


In [2]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.llms import OpenAI # Asegúrate de que esta importación sea correcta según tu setup
from langchain.chains import LLMChain
import sqlite3
import pandas as pd
from fuzzywuzzy import process

import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from keys import LLM

llm = LLM

sql_system_prompt = SystemMessagePromptTemplate.from_template(
    """Eres un asistente experto en bases de datos SQLite.

Tu tarea es generar **ÚNICAMENTE una query SQL válida**, sin explicaciones, sin texto adicional y sin formato markdown (como triples comillas ```, la palabra 'sql', o cualquier otro envoltorio), a partir de una pregunta del usuario.

Trabajas con una tabla llamada `comida`, el contenido de las tablas de texto viene en inglés, con las siguientes columnas. **¡IMPORTANTE! Cuando uses los nombres de las columnas que contienen puntos o guiones, DEBES ENVOLVERLOS ENTRE COMILLAS DOBLES (")** para que la consulta sea válida en SQLite.

- "category" (TEXT) - Categoría general del alimento. **Contiene una gran variedad de valores únicos.**
- "description" (TEXT) - Descripción completa del alimento y sus subcategorías (ej. 'Milk, human', 'Buttermilk, low fat (1%)').
- "alpha_carotene" (REAL) - Pigmento vegetal. Unidad: microgramos (mcg).
- "beta_carotene" (REAL) - Pigmento vegetal. Unidad: microgramos (mcg).
- "beta_cryptoxanthin" (REAL) - Pigmento vegetal. Unidad: microgramos (mcg).
- "carbohydrate" (REAL) - Azúcares y almidones. Unidad: gramos (g).
- "cholesterol" (REAL) - Presente solo en productos animales. Unidad: miligramos (mg).
- "choline" (REAL) - Nutriente esencial. Unidad: miligramos (mg).
- "fiber" (REAL) - Parte vegetal no digerible. Unidad: gramos (g).
- "lutein_and_zeaxanthin" (REAL) - Pigmento vegetal. Unidad: microgramos (mcg).
- "lycopene" (REAL) - Pigmento vegetal. Unidad: microgramos (mcg).
- "niacin" (REAL) - Vitamina B. Unidad: miligramos (mg).
- "protein" (REAL) - Esencial para tejidos. Unidad: gramos (g).
- "retinol" (REAL) - Vitamina A activa. Unidad: microgramos (mcg).
- "riboflavin" (REAL) - Vitamina B. Unidad: miligramos (mg).
- "selenium" (REAL) - Antioxidante. Unidad: microgramos (mcg).
- "sugar_total" (REAL) - Azúcares simples. Unidad: gramos (g).
- "thiamin" (REAL) - Vitamina B. Unidad: miligramos (mg).
- "water" (REAL) - Cantidad de agua. Unidad: gramos (g).
- "fat.monosaturated_fat" (REAL) - Grasas monoinsaturadas. Unidad: gramos (g).
- "fat.polysaturated_fat" (REAL) - Grasas poliinsaturadas. Unidad: gramos (g).
- "fat.saturated_fat" (REAL) - Grasas saturadas. Unidad: gramos (g).
- "fat.total_lipid" (REAL) - Suma de todas las grasas. Unidad: gramos (g).
- "major_minerals.calcium" (REAL) - Mineral. Unidad: miligramos (mg).
- "major_minerals.copper" (REAL) - Mineral. Unidad: miligramos (mg).
- "major_minerals.iron" (REAL) - Mineral. Unidad: miligramos (mg).
- "major_minerals.magnesium" (REAL) - Mineral. Unidad: miligramos (mg).
- "major_minerals.phosphorus" (REAL) - Mineral. Unidad: miligramos (mg).
- "major_minerals.potassium" (REAL) - Mineral. Unidad: miligramos (mg).
- "major_minerals.sodium" (REAL) - Mineral. Unidad: miligramos (mg).
- "major_minerals.zinc" (REAL) - Mineral. Unidad: miligramos (mg).
- "vitamins.vitamin_a_-_rae" (REAL) - Vitamina A. Unidad: microgramos (mcg).
- "vitamins.vitamin_b12" (REAL) - Vitamina B12. Unidad: microgramos (mcg).
- "vitamins.vitamin_b6" (REAL) - Vitamina B6. Unidad: miligramos (mg).
- "vitamins.vitamin_c" (REAL) - Vitamina C. Unidad: miligramos (mg).
- "vitamins.vitamin_e" (REAL) - Vitamina E. Unidad: miligramoscg).
- "vitamins.vitamin_k" (REAL) - Vitamina K. Unidad: microgramos (mcg).
- "calories" (REAL) - Calorías por 100g.

GUIDELINES:

**1. Cuando una columna específica (como calorías, proteína, etc.) sea solicitada, SIEMPRE incluye también las columnas "category" y "description" en la cláusula SELECT.**
**2. Para preguntas que solicitan una propiedad general (ej. "alimentos con X calorías", "alimentos con más Y"), enfócate en la condición numérica o de ordenación en el WHERE/ORDER BY. Solo usa filtros de "category" o "description" (LIKE) si la pregunta especifica un TIPO de alimento a buscar.**
**3. Devuelve ÚNICAMENTE una consulta SQL válida. No incluyas ningún texto extra, explicaciones o formato markdown (como triples comillas ```, la palabra 'sql', o cualquier otro envoltorio).**
**4. Para filtrar por alimentos específicos, usa la columna "description" con el operador LIKE y el texto del alimento CAPITALIZADO (Camel Case) en inglés, dado que la base de datos está en inglés, por ejemplo: LIKE '%Chicken Breast%' o LIKE '%Avocado%'.**
**5. Para filtrar por categorías generales de alimentos, usa la columna "category" con el operador '=' y el valor exacto de la categoría (en Camel Case) en inglés.**
**6. Recuerda que los nombres de las columnas que contienen puntos o guiones DEBEN ENVOLVERSE ENTRE COMILLAS DOBLES (") en la consulta SQL (ej. "fat.total_lipid", "vitamins.vitamin_a_-_rae").**
"""
)

sql_human_prompt = HumanMessagePromptTemplate.from_template(
    """{contexto}
    Pregunta: {pregunta}
    """
)

sql_prompt = ChatPromptTemplate.from_messages([
    sql_system_prompt,
    sql_human_prompt
])


sql_chain = LLMChain(llm=llm, prompt=sql_prompt)

C:\Users\asusg\AppData\Local\Temp\ipykernel_17868\2661594649.py:86: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sql_chain = LLMChain(llm=llm, prompt=sql_prompt)


### Filtrar las categorias en las que puede buscar

Ahora, antes de generar las busquedas, voy a guardar las categorias de comida.db para ofrecerle un contecto
mucho más completo al LLM

In [11]:
DATABASE_PATH = "../comida.db"

translator_system_prompt = SystemMessagePromptTemplate.from_template(
    "Traduce el siguiente término de alimento o categoría del español a su equivalente en inglés más común, adecuado para una consulta de base de datos. Responde ÚNICAMENTE con el término en inglés, sin explicaciones ni texto adicional."
)
translator_human_prompt = HumanMessagePromptTemplate.from_template("{term_spanish}")
translator_prompt = ChatPromptTemplate.from_messages([
    translator_system_prompt,
    translator_human_prompt
])

translator_chain = LLMChain(llm=llm, prompt=translator_prompt)

def get_unique_categories(db_path):
    """Carga todas las categorías únicas de la base de datos."""
    conn = sqlite3.connect(db_path)
    try:
        query = 'SELECT DISTINCT "category" FROM comida'
        categories_df = pd.read_sql_query(query, conn)
        return categories_df["category"].tolist()
    except Exception as e:
        print(f"Error al cargar categorías: {e}")
        return []
    finally:
        conn.close()

# Cargar las categorías al inicio de tu script/aplicación
# Esto se ejecutaría una sola vez
unique_categories = get_unique_categories(DATABASE_PATH)

In [32]:
unique_categories

['Milk',
 'Buttermilk',
 'Kefir',
 "Goat's milk",
 'Non-dairy milk',
 'Soy milk',
 'Almond milk',
 'Rice milk',
 'Coconut milk',
 'Yogurt',
 'Chipotle dip',
 'Dill dip',
 'Onion dip',
 'Ranch dip',
 'Spinach dip',
 'Tzatziki dip',
 'Vegetable dip',
 'Yogurt parfait',
 'Frozen yogurt',
 'Frozen yogurt sandwich',
 'Frozen yogurt bar',
 'Frozen yogurt cone',
 'Chocolate milk',
 'Hot chocolate / Cocoa',
 'Strawberry milk',
 'Eggnog',
 'Milk shake',
 'Milk shake with malt',
 'Licuado or Batido',
 'Fruit smoothie',
 'Fruit smoothie juice drink',
 'Chocolate milk drink',
 'Infant formula',
 'Whey',
 'Cocoa powder',
 'Chocolate beverage powder',
 'Strawberry beverage powder',
 'Cream',
 'Coffee creamer',
 'Whipped topping',
 'Sour cream',
 'Dip',
 'Ice cream',
 'Gelato',
 'Ice cream bar',
 'Ice cream candy bar',
 'Ice cream sandwich',
 'Ice cream cookie sandwich',
 'Ice cream cone',
 'Ice cream soda',
 'Ice cream sundae',
 'Banana split',
 'Light ice cream',
 'Soft serve',
 'Light ice cream sa

## NO EJECUTAR DE NUEVO
Traducir la variable categorias para que el LLM busque en categoria por esas variables en de inventarse el nombre

In [ ]:
# import time
# import ast

# translator_system_prompt_bulk = SystemMessagePromptTemplate.from_template(
#     """Eres un traductor experto. Tu tarea es traducir una lista de categorías de alimentos del inglés al español.
#     Para cada categoría, proporciona la traducción más común y útil para identificar ese tipo de alimento en una consulta en español.
#     Devuelve las traducciones como una lista de strings de Python, donde cada string es la traducción en español de una categoría.
#     Asegúrate de que la salida sea un string que represente una lista de Python válida, sin ningún texto adicional o explicaciones.

#     Ejemplo de entrada:
#     - "Milk"
#     - "Meats and Poultry"
#     - "Fresh Fruits"

#     Ejemplo de salida:
#     ["Leche", "Carnes y Aves", "Frutas Frescas"]

#     Asegúrate de manejar categorías compuestas correctamente y de usar la capitalización o el formato más natural para un nombre de alimento/categoría en español.
#     Si una categoría ya parece ser una palabra en español, devuélvela tal cual, a menos que haya una traducción más común.
#     """
# )
# translator_human_prompt_bulk = HumanMessagePromptTemplate.from_template("Traduce las siguientes categorías:\n{categories_list_english}")
# translator_prompt_bulk = ChatPromptTemplate.from_messages([
#     translator_system_prompt_bulk,
#     translator_human_prompt_bulk
# ])
# translator_chain_bulk = LLMChain(llm=llm, prompt=translator_prompt_bulk)

# # --- 3. Función para dividir lista en lotes (chunks) ---
# def chunk_list(data, chunk_size):
#     for i in range(0, len(data), chunk_size):
#         yield data[i:i + chunk_size]
        

# all_translated_categories_es = [] # Lista para recolectar todas las traducciones
    
# BATCH_SIZE = 100 
# SLEEP_TIME_SECONDS = 5 

# print(f"Total de categorías a traducir: {len(unique_categories)}")
# print(f"Procesando en lotes de {BATCH_SIZE} con {SLEEP_TIME_SECONDS} segundos de espera entre lotes.")

# batches = list(chunk_list(unique_categories, BATCH_SIZE))
# total_batches = len(batches)

# for i, batch in enumerate(batches):
#     print(f"\nProcesando lote {i+1}/{total_batches} (elementos {i*BATCH_SIZE + 1} a {(i+1)*BATCH_SIZE} de {len(unique_categories)})...")
    
#     # Formatear el lote actual para el prompt del LLM
#     categories_for_prompt = "\n".join([f"- \"{cat}\"" for cat in batch])

#     try:
#         start_time = time.time()
#         # Llamada al LLM para el lote actual
#         translated_categories_str = translator_chain_bulk.run(
#             categories_list_english=categories_for_prompt
#         ).strip()
#         end_time = time.time()
#         print(f"Llamada al LLM del lote {i+1} completada en {end_time - start_time:.2f} segundos.")

#         # Parsear el string de Python list y añadirlo a la lista global
#         current_batch_translations = ast.literal_eval(translated_categories_str)
#         all_translated_categories_es.extend(current_batch_translations)
#         print(f"Traducciones añadidas del lote {i+1}: {len(current_batch_translations)}.")

#     except Exception as e:
#         print(f"ERROR al traducir lote {i+1}: {e}")
#         print("Saltando este lote. El proceso continuará con el siguiente. Revisa el error y considera reintentar este lote manualmente.")
#         # Si un lote falla, no detenemos todo, pero guardaremos lo que se ha podido hacer.
#         # Puedes implementar una lógica más sofisticada aquí si necesitas reintentos.

#     # Pausar entre lotes para evitar límites de tasa de la API del LLM
#     if i < total_batches - 1:
#         print(f"Esperando {SLEEP_TIME_SECONDS} segundos antes del siguiente lote...")
#         time.sleep(SLEEP_TIME_SECONDS)

Total de categorías a traducir: 2380
Procesando en lotes de 100 con 5 segundos de espera entre lotes.

Procesando lote 1/24 (elementos 1 a 100 de 2380)...


C:\Users\asusg\AppData\Local\Temp\ipykernel_20592\3058728298.py:55: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  translated_categories_str = translator_chain_bulk.run(


Llamada al LLM del lote 1 completada en 5.90 segundos.
Traducciones añadidas del lote 1: 100.
Esperando 5 segundos antes del siguiente lote...

Procesando lote 2/24 (elementos 101 a 200 de 2380)...
Llamada al LLM del lote 2 completada en 7.03 segundos.
Traducciones añadidas del lote 2: 99.
Esperando 5 segundos antes del siguiente lote...

Procesando lote 3/24 (elementos 201 a 300 de 2380)...
Llamada al LLM del lote 3 completada en 7.37 segundos.
Traducciones añadidas del lote 3: 100.
Esperando 5 segundos antes del siguiente lote...

Procesando lote 4/24 (elementos 301 a 400 de 2380)...
Llamada al LLM del lote 4 completada en 10.82 segundos.
Traducciones añadidas del lote 4: 100.
Esperando 5 segundos antes del siguiente lote...

Procesando lote 5/24 (elementos 401 a 500 de 2380)...
Llamada al LLM del lote 5 completada en 10.16 segundos.
Traducciones añadidas del lote 5: 100.
Esperando 5 segundos antes del siguiente lote...

Procesando lote 6/24 (elementos 501 a 600 de 2380)...
Llamada a

Categorias traducidas guardadas en un archivo JSON para no ejecutar cada vez

In [ ]:
# import json

# file_name = "specific_food_translations.json"
# file_path = f"./data/csv/{file_name}"
# with open(file_path, 'w', encoding='utf-8') as f:
#     json.dump(all_translated_categories_es, f, ensure_ascii=False, indent=4)
# print(f"Traducciones guardadas en {file_path}")

Traducciones guardadas en ./data/csv/specific_food_translations.json


In [1]:
import json
import os

file_name = "specific_food_translations.json"
file_path = f"../data/csv/{file_name}"
all_translated_categories_es = []

if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        all_translated_categories_es = json.load(f)
    print(f"Traducciones cargadas desde {file_path} ({len(all_translated_categories_es)} términos).")
else:
    print(f"Archivo {file_path} no encontrado. Generando traducciones...")

Traducciones cargadas desde ../data/csv/specific_food_translations.json (2379 términos).


In [9]:
def resolver_sql(pregunta):
    contexto_pistas = ""
    pregunta_normalizada = pregunta.lower()

    # 1. Identificar si es una pregunta general o específica
    # Una forma simple es buscar si la pregunta contiene términos de "propiedad general"
    # que no suelen ir acompañados de un alimento específico.
    general_property_keywords = [
        "calorias", "carbohidratos", "colesterol", "colina", "fibra", "niacina", 
        "proteina", "retinol", "riboflavina", "selenio", "azucar total", "tiamina", 
        "agua", "grasa monosaturada", "grasa polisaturada", "grasa saturada", 
        "lipido total", "calcio", "cobre", "hierro", "magnesio", "fosforo", 
        "potasio", "sodio", "zinc", "vitamina a", "vitamina b12", "vitamina b6", 
        "vitamina c", "vitamina e", "vitamina k", "alfa caroteno", "beta caroteno", 
        "beta criptoxantina", "luteina y zeaxantina", "licopeno"
    ]    # Si la pregunta contiene alguna de estas palabras clave Y NO contiene un nombre de alimento específico
    # (como aguacate, pollo, etc.), la consideramos "general".
    
    # Simplificamos la detección de pregunta específica basándonos en la aparición de nombres de alimentos traducibles
    # Si la traducción de la frase completa contiene un nombre de alimento claro, es específica.
    
    # Intenta traducir la pregunta completa para obtener un término que pueda usarse en `LIKE`
    translated_phrase = ""
    try:
        translated_phrase = translator_chain.run(term_spanish=pregunta).strip()
        print(f"Frase traducida para búsqueda: '{translated_phrase}'")
    except Exception as e:
        print(f"Error al traducir la pregunta completa: {e}")

    # Lista de nombres de alimentos clave en inglés (CamelCase)
    # que, si están en la pregunta traducida, implican una búsqueda específica.
    specific_food_translations = unique_categories 
    
    is_specific_food_query = False
    for food_term in specific_food_translations:
        if food_term.lower() in translated_phrase.lower():
            is_specific_food_query = True
            break
            
    # Si la pregunta es sobre una propiedad general (ej. "menos de X calorías")
    # Y no se detecta un alimento específico en la pregunta, la tratamos como general.
    # Esto ayuda a que no se inyecten filtros de categoría/descripción irrelevantes.
    is_general_numeric_query = any(keyword in pregunta_normalizada for keyword in general_property_keywords) and not is_specific_food_query
    
    pregunta_procesada = pregunta

    # --- Lógica de fuzzy matching y determinación de pistas ---
    matched_categories_for_llm = []
    matched_descriptions_for_llm = []

    # Búsqueda en CATEGORIES
    # Solo añadir categorías si es una consulta sobre un alimento específico
    if is_specific_food_query:
        for cat_db in unique_categories:
            # Coincidencia directa del nombre de la categoría con la pregunta traducida o original
            if cat_db.lower() in translated_phrase.lower() or cat_db.lower() in pregunta_normalizada:
                if cat_db not in matched_categories_for_llm:
                    matched_categories_for_llm.append(cat_db)
            
            # Fuzzy matching para categorías (puedes ajustar el umbral)
            cat_display_name_for_fuzzy = ' '.join([word.capitalize() for word in cat_db.replace('.', ' ').split()])
            matches_cat = process.extract(translated_phrase.lower(), [cat_display_name_for_fuzzy.lower()], limit=1, scorer=process.fuzz.token_set_ratio)
            if matches_cat and matches_cat[0][1] > 90: # Umbral alto para evitar falsos positivos como "Lime"
                 if cat_db not in matched_categories_for_llm:
                    matched_categories_for_llm.append(cat_db)
                    
    # Las descripciones relevantes provienen de la traducción de la frase completa
    # y los términos específicos si se identificaron.
    # Solo añadir descripciones LIKE si es una consulta de alimento específico.
    if is_specific_food_query and translated_phrase:
        # Añade la traducción CamelCase de la frase completa como una pista de descripción
        # si contiene un término de alimento específico y no es una propiedad genérica.
        camel_case_translated_phrase = ' '.join([word.capitalize() for word in translated_phrase.split()])
        if camel_case_translated_phrase not in matched_descriptions_for_llm:
            matched_descriptions_for_llm.append(camel_case_translated_phrase)

    # --- Construcción del contexto de pistas inteligentemente ---
    contexto_pistas = ""
    
    if is_specific_food_query:
        if matched_categories_for_llm:
            matched_categories_unique = list(set(matched_categories_for_llm))
            # Opcional: Limitar el número de categorías para no sobrecargar el prompt
            if len(matched_categories_unique) > 3: 
                matched_categories_unique = matched_categories_unique[:3] 
            contexto_pistas += f"El usuario podría referirse a las siguientes categorías: {', '.join(matched_categories_unique)}. "
            
        if matched_descriptions_for_llm:
            matched_descriptions_unique = list(set(matched_descriptions_for_llm))
            # Opcional: Limitar el número de descripciones para no sobrecargar el prompt
            if len(matched_descriptions_unique) > 5:
                 matched_descriptions_unique = matched_descriptions_unique[:5] 
            # Damos una pista fuerte al LLM para que use estos términos en LIKE
            contexto_pistas += f"El usuario podría referirse a alimentos con estas descripciones (usa LIKE con los términos ya en inglés CamelCase): {', '.join(matched_descriptions_unique)}. "
    
    # Para preguntas generales, no inyectamos pistas de categoría/descripción de alimentos,
    # el LLM debe usar las propiedades numéricas directamente.
    # La directriz 2 del prompt de SQL es crucial aquí.
    # No es necesario un 'else' explícito, si las listas están vacías, contexto_pistas quedará vacío.

    if contexto_pistas:
        print(f"Pistas inyectadas: {contexto_pistas}")
    else:
        print("No se inyectaron pistas de categoría/descripción.")


    # Paso 1: usar el LLM para generar la SQL a partir de la pregunta, pasando el contexto y la pregunta procesada
    consulta_sql = sql_chain.run(
        pregunta=pregunta, # Usamos la pregunta original para el LLM de SQL
        contexto=contexto_pistas # Las pistas se inyectan aquí
    ).strip()
    
    # Esto actúa como una salvaguarda en caso de que el LLM aún intente añadir el formato.
    if consulta_sql.startswith('```sql\n') and consulta_sql.endswith('\n```'):
        consulta_sql = consulta_sql[len('```sql\n'):-len('\n```')].strip()
    elif consulta_sql.startswith('```') and consulta_sql.endswith('```'):
        consulta_sql = consulta_sql[len('```'):-len('```')].strip()
        
    print(f"SQL generada:\n{consulta_sql}\n")

    # Paso 2: ejecutar la consulta sobre la base de datos SQLite
    try:
        conn = sqlite3.connect(DATABASE_PATH)
        resultado = pd.read_sql_query(consulta_sql, conn)
        print(resultado)
        return resultado
    except Exception as e:
        return f"Error al ejecutar la consulta:\n{e}"
    finally:
        if conn:
            conn.close()


In [13]:
import time

In [20]:
preguntas = ["¿Qué alimentos tienen menos de 10 calorias, dime los 5 con menos?",
            "¿Cuántas calorías tiene un aguacate?",
            "¿Cuántos gramos de proteína hay en 100g de pechuga de pollo? Devuelve 5",
            "¿Cuál es el contenido de hierro en las lentejas? Devuelve 5",
            "Dime los microgramos de Vitamina B12 que tiene el salmón. Devuelve 5",
            "Enumera los 5 alimentos con más potasio de tu base de datos.",
            "¿Qué tiene más grasa saturada, el queso cheddar o la mantequilla?"
]

for pregunta in preguntas:
    print(f"Pregunta: {pregunta}")
    resolver_sql(pregunta)
    time.sleep(1)

Pregunta: ¿Qué alimentos tienen menos de 10 calorias, dime los 5 con menos?
Frase traducida para búsqueda: 'Low-calorie foods'
No se inyectaron pistas de categoría/descripción.
SQL generada:
SELECT "category", "description", "calories" FROM comida WHERE "calories" < 10 ORDER BY "calories" ASC LIMIT 5;

  category                                        description  calories
0      Tea               Tea, hot, leaf, green, decaffeinated       0.0
1      Tea             Tea, iced, instant, green, unsweetened       0.0
2      Tea                                 Tea, hot, hibiscus       0.0
3      Tea  Tea, iced, brewed, green, decaffeinated, unswe...       0.0
4      Tea             Tea, iced, bottled, black, unsweetened       0.0
Pregunta: ¿Cuántas calorías tiene un aguacate?
Frase traducida para búsqueda: 'avocado'
Pistas inyectadas: El usuario podría referirse a las siguientes categorías: Avocado, Avocado dressing. El usuario podría referirse a alimentos con estas descripciones (usa LIKE

In [19]:
preguntas = [
    "¿Qué alimentos tienen más de 70 gramos de carbohidratos y menos de 50 gramos de grasa saturada? Devuelve 5",
    "Que alimentos tienen menos grasas saturadas y más proteinas, devuelve 5",
    "¿Cuántas calorías tiene una manzana? Devuelve 2"
]

for pregunta in preguntas:
    print(f"Pregunta: {pregunta}")
    resolver_sql(pregunta)
    time.sleep(1)

Pregunta: ¿Qué alimentos tienen más de 70 gramos de carbohidratos y menos de 50 gramos de grasa saturada? Devuelve 5
Frase traducida para búsqueda: 'carbohydrates'
No se inyectaron pistas de categoría/descripción.
SQL generada:
SELECT "category", "description", "carbohydrate", "fat.saturated_fat" FROM comida WHERE "carbohydrate" > 70 AND "fat.saturated_fat" < 50 LIMIT 5;

                     category  \
0                        Whey   
1   Chocolate beverage powder   
2                        Milk   
3  Strawberry beverage powder   
4              Coffee creamer   

                                         description  carbohydrate  \
0                                   Whey, sweet, dry         74.46   
1  Chocolate beverage powder, dry mix, not recons...         90.28   
2           Milk, malted, dry mix, not reconstituted         92.96   
3  Strawberry beverage powder, dry mix, not recon...         99.10   
4                   Coffee creamer, powder, flavored         75.42   

   fa

### Routing por inferencia con LLM

In [ ]:
from keys import LLM
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

llm = LLM 

# PROMPT PARA ENRUTAMIENTO (routing)
# Este prompt instruye al modelo para que, dada una pregunta en lenguaje natural, clasifique si debe resolverse con una búsqueda estructurada (SQL)
# o con una búsqueda semántica (RAG).
routing_prompt = ChatPromptTemplate.from_template(
    """Eres un sistema experto que enruta una pregunta de nutrición a la fuente de datos correcta.

Clasifica cada pregunta en una de estas dos opciones:
- "sql" → si la pregunta es sobre la composición nutricional específica de uno o más alimentos. Esto incluye consultas sobre cantidades exactas de nutrientes (ej. gramos de proteína, miligramos de vitamina C), calorías, minerales o grasas en un alimento. También se incluyen comparaciones directas o rankings de alimentos basados en estos datos numéricos.
- "rag" → si la pregunta busca una explicación, recomendación, o información contextual sobre nutrición, suplementación o fitness. Esto abarca preguntas sobre la función de un suplemento (ej. "¿para qué sirve la creatina?"), beneficios, dosis recomendadas, dietas para objetivos específicos (ej. "dieta para diabetes tipo 2"), o consejos basados en un perfil de usuario (ej. "suplementos para un corredor").

Responde únicamente con la palabra: sql o rag.

Pregunta: {pregunta}
"""
)


#Aquí conectamos el prompt anterior con el modelo de lenguaje (llm).
# El resultado es una cadena que puede ejecutarse para obtener el tipo de consulta (sql / mongo).
routing_chain = LLMChain(llm=llm, prompt=routing_prompt)



# Esta función toma una pregunta del usuario y usa el LLM para decidir
# qué tipo de base de datos debe consultarse.
# La salida puede ser "sql", "rag", o un mensaje de error.

def router(pregunta, routing_chain):
    try:
        tipo = routing_chain.invoke({"pregunta": pregunta})["text"].strip().lower()
    except Exception as e:
        return f"Error al decidir la ruta: {e}"

    print(f"Tipo de consulta inferido: {tipo}")

    if tipo == "sql":
        return tipo
        #return resolver_sql(pregunta)
    elif tipo == "rag":
        return tipo
        #return resolver_mongo(pregunta)
    else:
        return f"Ruta desconocida: '{tipo}'"

C:\Users\asusg\AppData\Local\Temp\ipykernel_7624\1159787679.py:27: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  routing_chain = LLMChain(llm=llm, prompt=routing_prompt)


### Routing entre BBDD y RAG

In [3]:
import time

In [4]:
preguntas_sql = [
    # --- Preguntas claramente SQL ---
    "¿Cuántos gramos de proteína hay en 100g de pechuga de pollo?",
    "¿Cuál es el contenido de hierro en las lentejas?",
    "Dime los microgramos de Vitamina B12 que tiene el salmón.",
    "Enumera los 5 alimentos con más potasio de tu base de datos.",
    "¿Qué tiene más grasa saturada, el queso cheddar o la mantequilla?",
    "¿Cuántas calorías tiene un aguacate?",
]

for pregunta in preguntas_sql:
    tipo = router(pregunta)
    print(f"[{tipo.upper()}] → {pregunta}")
    print("\n")
    time.sleep(1)  # espera 1 segundo entre llamadas

Tipo de consulta inferido: sql
[SQL] → ¿Cuántos gramos de proteína hay en 100g de pechuga de pollo?


Tipo de consulta inferido: sql
[SQL] → ¿Cuál es el contenido de hierro en las lentejas?


Tipo de consulta inferido: sql
[SQL] → Dime los microgramos de Vitamina B12 que tiene el salmón.


Tipo de consulta inferido: sql
[SQL] → Enumera los 5 alimentos con más potasio de tu base de datos.


Tipo de consulta inferido: sql
[SQL] → ¿Qué tiene más grasa saturada, el queso cheddar o la mantequilla?


Tipo de consulta inferido: sql
[SQL] → ¿Cuántas calorías tiene un aguacate?




In [5]:
preguntas_rag = [
    # --- Preguntas claramente RAG ---
    "¿Para qué sirve la creatina y cómo se debe tomar?",
    "Explica los beneficios del omega-3 en el rendimiento deportivo.",
    "¿Qué tipo de proteína es mejor para ganar masa muscular?",
    "Soy una mujer de 50 años que hace pesas, ¿qué suplementos me recomiendas para mantener la masa ósea?",
    "¿Qué evidencia hay sobre el uso de la ashwagandha para reducir el cortisol?",
    "¿Cuáles son los principios de una dieta antiinflamatoria?",
]

for pregunta in preguntas_rag:
    tipo = router(pregunta)
    print(f"[{tipo.upper()}] → {pregunta}")
    print("\n")
    time.sleep(1)  # espera 1 segundo entre llamadas

Tipo de consulta inferido: rag
[RAG] → ¿Para qué sirve la creatina y cómo se debe tomar?


Tipo de consulta inferido: rag
[RAG] → Explica los beneficios del omega-3 en el rendimiento deportivo.


Tipo de consulta inferido: rag
[RAG] → ¿Qué tipo de proteína es mejor para ganar masa muscular?


Tipo de consulta inferido: rag
[RAG] → Soy una mujer de 50 años que hace pesas, ¿qué suplementos me recomiendas para mantener la masa ósea?


Tipo de consulta inferido: rag
[RAG] → ¿Qué evidencia hay sobre el uso de la ashwagandha para reducir el cortisol?


Tipo de consulta inferido: rag
[RAG] → ¿Cuáles son los principios de una dieta antiinflamatoria?




In [6]:

preguntas_sql_rag = [
    # --- Pares de preguntas (SQL vs RAG) ---
    "¿Qué tiene más hierro, 100g de espinacas o 100g de carne roja?", #SQL
    "¿Por qué es importante el hierro para prevenir la fatiga en atletas?", #RAG
    "Lista 3 tipos de pescado ordenados por su contenido de proteína.", #SQL
    "¿Es mejor tomar la proteína antes o después de entrenar para la recuperación muscular?", #RAG
    "¿Cuántos gramos de azúcar total contiene una manzana?", #SQL
    "Soy diabético tipo 2, ¿qué tipo de carbohidratos debería priorizar en mi dieta?", #RAG
    "¿Cuál es el desglose de grasas (saturadas, mono, poli) del aceite de oliva?", #SQL
    "¿Qué diferencia hay para la salud entre las grasas saturadas y las insaturadas?",  #RAG
]

for pregunta in preguntas_sql_rag:
    tipo = router(pregunta)
    print(f"[{tipo.upper()}] → {pregunta}")
    print("\n")
    time.sleep(1)  # espera 1 segundo entre llamadas

Tipo de consulta inferido: sql
[SQL] → ¿Qué tiene más hierro, 100g de espinacas o 100g de carne roja?


Tipo de consulta inferido: rag
[RAG] → ¿Por qué es importante el hierro para prevenir la fatiga en atletas?


Tipo de consulta inferido: sql
[SQL] → Lista 3 tipos de pescado ordenados por su contenido de proteína.


Tipo de consulta inferido: rag
[RAG] → ¿Es mejor tomar la proteína antes o después de entrenar para la recuperación muscular?


Tipo de consulta inferido: sql
[SQL] → ¿Cuántos gramos de azúcar total contiene una manzana?


Tipo de consulta inferido: rag
[RAG] → Soy diabético tipo 2, ¿qué tipo de carbohidratos debería priorizar en mi dieta?


Tipo de consulta inferido: sql
[SQL] → ¿Cuál es el desglose de grasas (saturadas, mono, poli) del aceite de oliva?


Tipo de consulta inferido: rag
[RAG] → ¿Qué diferencia hay para la salud entre las grasas saturadas y las insaturadas?




In [7]:

preguntas_trampa = [
    # --- Preguntas trampa o en el límite ---
    "Quiero saber cuánta fibra tiene la avena y por qué es buena para la digestión.",
    "¿Es saludable el aguacate?",
    "He oído que el magnesio es bueno. ¿Qué alimentos lo contienen y qué suplemento me recomiendas para dormir mejor?"
]

for pregunta in preguntas_trampa:
    tipo = router(pregunta)
    print(f"[{tipo.upper()}] → {pregunta}")
    print("\n")
    time.sleep(1)  # espera 1 segundo entre llamadas

Tipo de consulta inferido: sql
[SQL] → Quiero saber cuánta fibra tiene la avena y por qué es buena para la digestión.


Tipo de consulta inferido: rag
[RAG] → ¿Es saludable el aguacate?


Tipo de consulta inferido: rag
[RAG] → He oído que el magnesio es bueno. ¿Qué alimentos lo contienen y qué suplemento me recomiendas para dormir mejor?




### RESULTADOS

#### --- Preguntas claramente SQL ---
"¿Cuántos gramos de proteína hay en 100g de pechuga de pollo?",
"¿Cuál es el contenido de hierro en las lentejas?",
"Dime los microgramos de Vitamina B12 que tiene el salmón.",
"Enumera los 5 alimentos con más potasio de tu base de datos.",
"¿Qué tiene más grasa saturada, el queso cheddar o la mantequilla?",
"¿Cuántas calorías tiene un aguacate?",

#### --- Preguntas claramente RAG ---
"¿Para qué sirve la creatina y cómo se debe tomar?",
"Explica los beneficios del omega-3 en el rendimiento deportivo.",
"¿Qué tipo de proteína es mejor para ganar masa muscular?",
"Soy una mujer de 50 años que hace pesas, ¿qué suplementos me recomiendas para mantener la masa ósea?",
"¿Qué evidencia hay sobre el uso de la ashwagandha para reducir el cortisol?",
"¿Cuáles son los principios de una dieta antiinflamatoria?",

#### --- Pares de preguntas (SQL vs RAG) ---
"¿Qué tiene más hierro, 100g de espinacas o 100g de carne roja?" --> SQL,
"¿Por qué es importante el hierro para prevenir la fatiga en atletas?" --> RAG,
"Lista 3 tipos de pescado ordenados por su contenido de proteína." --> SQL,
"¿Es mejor tomar la proteína antes o después de entrenar para la recuperación muscular?" --> RAG,
"¿Cuántos gramos de azúcar total contiene una manzana?" --> SQL,
"Soy diabético tipo 2, ¿qué tipo de carbohidratos debería priorizar en mi dieta?" --> RAG,
"¿Cuál es el desglose de grasas (saturadas, mono, poli) del aceite de oliva?" --> SQL,
"¿Qué diferencia hay para la salud entre las grasas saturadas y las insaturadas?" --> RAG,

#### --- Preguntas trampa o en el límite ---
"Quiero saber cuánta fibra tiene la avena y por qué es buena para la digestión."

Resultado esperado: rag. Aunque la primera parte es sql, la segunda ("y por qué es buena") requiere una explicación que solo el RAG puede dar. Un buen sistema debería priorizar la pregunta más compleja.
,

"¿Es saludable el aguacate?"

Resultado esperado: rag. La palabra "saludable" es subjetiva. Para responderla, el sistema necesita sintetizar la información nutricional (grasas monoinsaturadas, vitaminas, etc., datos que están en la SQL) y explicar por qué esos componentes son beneficiosos, una tarea propia del RAG.
,


"He oído que el magnesio es bueno. ¿Qué alimentos lo contienen y qué suplemento me recomiendas para dormir mejor?"

Resultado esperado: rag. La intención principal es una recomendación de suplemento para un fin específico ("dormir mejor"), lo cual es una consulta para el RAG.


## Paso final 

Hasta ahora ya se tiene el Routing, el sql y el rag. Queda devolver la respuesta personalizada para el usuario.

```scss
[Pregunta del usuario]
        ↓
[LLM → Routing: sql / mongo]
        ↓
["sql"] → resolver_sql()
OR
["rag"] → resolver_mongo()
        ↓
[Contexto recuperado (DataFrame o texto)]
        ↓
[LLM: genera una respuesta final personalizada]
        ↓
[Respuesta final para el usuario]
```